# Day4-1 : VGG, RESNET model simple Implementation with CIFAR10 dataset (45 min)
#### 이번 실습에서는 CNN 아키텍쳐 중 가장 대표적인 VGG과 RESNET 모델을 간단히 구현해보고 CIFAR-10 데이터셋에 대해 학습 및 Inference를 해볼 계획입니다.

## Example 1) VGG16 모델 구현
- [doc] (https://arxiv.org/pdf/1409.1556.pdf)

![image.png](http://drive.google.com/uc?id=1E6MVIcFCsImwWQGOhC-8KUQqAe2W_I28)

![image.png](http://drive.google.com/uc?id=1jT9jhqMzEaHoma5xvro5jf6PFOq7FlLJ)

![image.png](http://drive.google.com/uc?id=17DgT11woHwXACEGOfvkVd8pYmxRI7vQl)

In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from torchvision import transforms, datasets, utils
from torchsummary import summary
import matplotlib.pyplot as plt

# DEVICE 설정
USE_CUDA = torch.cuda.is_available()
DEVICE = torch.device("cuda" if USE_CUDA else "cpu")

# Parameter 설정
EPOCHS = 10
BATCH_SIZE = 64
LR = 0.0001

# Transform 설정
transform_CIFAR10 = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize(mean=(0.485, 0.456, 0.406), std=(0.229, 0.224, 0.225)),
])

# Dataset 설정
train_dataset = datasets.CIFAR10(root = '../data',
                                         train = True,
                                         download = True,
                                         transform = transform_CIFAR10)

test_dataset = datasets.CIFAR10(root = '../data',
                                train = False,
                                download = True,
                                transform = transform_CIFAR10)

train_loader = torch.utils.data.DataLoader(dataset = train_dataset,
                                           batch_size = BATCH_SIZE,
                                           shuffle=True)

test_loader = torch.utils.data.DataLoader(dataset = test_dataset,
                                          batch_size = BATCH_SIZE,
                                          shuffle=False)

100%|██████████| 170498071/170498071 [00:13<00:00, 13021840.28it/s]


Extracting ../data/cifar-10-python.tar.gz to ../data
Files already downloaded and verified


In [5]:
# Model 구현
class Custom_VGG(nn.Module):
    def __init__(self):
        super(Custom_VGG, self).__init__()
        ########################################## Complete This Code~!

        # Conv layer
        self.conv1_1 = nn.Conv2d(3, 64, 3, 1, 1)
        self.conv1_2 = nn.Conv2d(64, 64, 3, 1, 1)

        self.conv2_1 = nn.Conv2d(64, 128, 3, 1, 1)
        self.conv2_2 = nn.Conv2d(128, 128, 3, 1, 1)

        self.conv3_1 = nn.Conv2d(128, 256, 3, 1, 1)
        self.conv3_2 = nn.Conv2d(256, 256, 3, 1, 1)
        self.conv3_3 = nn.Conv2d(256, 256, 3, 1, 1)

        self.conv4_1 = nn.Conv2d(256, 512, 3, 1, 1)
        self.conv4_2 = nn.Conv2d(512, 512, 3, 1, 1)
        self.conv4_3 = nn.Conv2d(512, 512, 3, 1, 1)

        self.conv5_1 = nn.Conv2d(512, 512, 3, 1, 1)
        self.conv5_2 = nn.Conv2d(512, 512, 3, 1, 1)
        self.conv5_3 = nn.Conv2d(512, 512, 3, 1, 1)

        self.maxpool = nn.MaxPool2d(kernel_size=2)

        self.adaptive_avg_pool = nn.AdaptiveAvgPool2d(7)

        self.fc1 = nn.Linear(512 * 7 * 7, 4096)
        self.fc2 = nn.Linear(4096, 4096)
        self.fc3 = nn.Linear(4096, 10)

        self.dropout = nn.Dropout(p=0.2)
        self.relu = nn.ReLU(inplace=True)


        ########################################## Complete This Code~!

    def forward(self, x):
        ########################################## Complete This Code~!

        x = self.relu(self.conv1_1(x))
        x = self.relu(self.conv1_2(x))
        x = self.maxpool(x)

        x = self.relu(self.conv2_1(x))
        x = self.relu(self.conv2_2(x))
        x = self.maxpool(x)

        x = self.relu(self.conv3_1(x))
        x = self.relu(self.conv3_2(x))
        x = self.relu(self.conv3_3(x))
        x = self.maxpool(x)

        x = self.relu(self.conv4_1(x))
        x = self.relu(self.conv4_2(x))
        x = self.relu(self.conv4_3(x))
        x = self.maxpool(x)

        x = self.relu(self.conv5_1(x))
        x = self.relu(self.conv5_2(x))
        x = self.relu(self.conv5_3(x))
        x = self.maxpool(x)

        x = self.adaptive_avg_pool(x)
        x = x.view(-1, 512 * 7 * 7)

        x = self.dropout(self.relu(self.fc1(x)))
        x = self.dropout(self.relu(self.fc2(x)))
        x = self.fc3(x)


        ########################################## Complete This Code~!
        return x

model = Custom_VGG().to(DEVICE)
summary(model, (3,32,32))

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1           [-1, 64, 32, 32]           1,792
              ReLU-2           [-1, 64, 32, 32]               0
            Conv2d-3           [-1, 64, 32, 32]          36,928
              ReLU-4           [-1, 64, 32, 32]               0
         MaxPool2d-5           [-1, 64, 16, 16]               0
            Conv2d-6          [-1, 128, 16, 16]          73,856
              ReLU-7          [-1, 128, 16, 16]               0
            Conv2d-8          [-1, 128, 16, 16]         147,584
              ReLU-9          [-1, 128, 16, 16]               0
        MaxPool2d-10            [-1, 128, 8, 8]               0
           Conv2d-11            [-1, 256, 8, 8]         295,168
             ReLU-12            [-1, 256, 8, 8]               0
           Conv2d-13            [-1, 256, 8, 8]         590,080
             ReLU-14            [-1, 25

In [6]:
from torchvision import models
model_import = models.vgg16(pretrained=False, num_classes=10).to(DEVICE)
summary(model_import, (3, 32, 32))

/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=None`.
  warnings.warn(msg)


----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1           [-1, 64, 32, 32]           1,792
              ReLU-2           [-1, 64, 32, 32]               0
            Conv2d-3           [-1, 64, 32, 32]          36,928
              ReLU-4           [-1, 64, 32, 32]               0
         MaxPool2d-5           [-1, 64, 16, 16]               0
            Conv2d-6          [-1, 128, 16, 16]          73,856
              ReLU-7          [-1, 128, 16, 16]               0
            Conv2d-8          [-1, 128, 16, 16]         147,584
              ReLU-9          [-1, 128, 16, 16]               0
        MaxPool2d-10            [-1, 128, 8, 8]               0
           Conv2d-11            [-1, 256, 8, 8]         295,168
             ReLU-12            [-1, 256, 8, 8]               0
           Conv2d-13            [-1, 256, 8, 8]         590,080
             ReLU-14            [-1, 25

In [8]:
# Optimizer 설정
optimizer = optim.Adam(model.parameters(), lr=LR)
# optimizer = optim.SGD(model.parameters(), lr=0.01, momentum=0.5)

In [9]:
# Train 구현
def train_one_epoch(model, train_loader, optimizer, epoch):
    model.train()
    for batch_idx, (data, target) in enumerate(train_loader):
        data, target = data.to(DEVICE), target.to(DEVICE)
        optimizer.zero_grad()
        output = model(data)
        loss = F.cross_entropy(output, target)
        loss.backward()
        optimizer.step()

        if batch_idx % 200 == 0:
            print('Train Epoch: {} [{}/{} ({:.0f}%)]\tLoss: {:.6f}'.format(
                epoch, batch_idx * len(data), len(train_loader.dataset),
                100. * batch_idx / len(train_loader), loss.item()))

# Evaluation 구현
def evaluate(model, test_loader):
    model.eval()
    test_loss = 0
    correct = 0
    with torch.no_grad():
        for data, target in test_loader:
            data, target = data.to(DEVICE), target.to(DEVICE)
            output = model(data)

            # 배치 오차를 합산
            test_loss += F.cross_entropy(output, target,
                                         reduction='sum').item()

            # 가장 높은 값을 가진 인덱스가 바로 예측값
            pred = output.max(1, keepdim=True)[1]
            correct += pred.eq(target.view_as(pred)).sum().item()

    test_loss /= len(test_loader.dataset)
    test_accuracy = 100. * correct / len(test_loader.dataset)
    return test_loss, test_accuracy

for epoch in range(1, EPOCHS + 1):
    train_one_epoch(model, train_loader, optimizer, epoch)
    test_loss, test_accuracy = evaluate(model, test_loader)

    print('[{}] Test Loss: {:.4f}, Accuracy: {:.2f}%'.format(
          epoch, test_loss, test_accuracy))

Train Epoch: 1 [0/50000 (0%)]	Loss: 2.301356
Train Epoch: 1 [12800/50000 (26%)]	Loss: 2.111874
Train Epoch: 1 [25600/50000 (51%)]	Loss: 1.756096
Train Epoch: 1 [38400/50000 (77%)]	Loss: 1.729194
[1] Test Loss: 1.6864, Accuracy: 32.54%
Train Epoch: 2 [0/50000 (0%)]	Loss: 1.715979
Train Epoch: 2 [12800/50000 (26%)]	Loss: 1.651258
Train Epoch: 2 [25600/50000 (51%)]	Loss: 1.572220
Train Epoch: 2 [38400/50000 (77%)]	Loss: 1.422531
[2] Test Loss: 1.3848, Accuracy: 46.67%
Train Epoch: 3 [0/50000 (0%)]	Loss: 1.285518
Train Epoch: 3 [12800/50000 (26%)]	Loss: 1.354075
Train Epoch: 3 [25600/50000 (51%)]	Loss: 1.316396
Train Epoch: 3 [38400/50000 (77%)]	Loss: 1.277216
[3] Test Loss: 1.2320, Accuracy: 55.39%
Train Epoch: 4 [0/50000 (0%)]	Loss: 1.246620
Train Epoch: 4 [12800/50000 (26%)]	Loss: 0.946330
Train Epoch: 4 [25600/50000 (51%)]	Loss: 0.911426
Train Epoch: 4 [38400/50000 (77%)]	Loss: 1.203128
[4] Test Loss: 0.9447, Accuracy: 66.40%
Train Epoch: 5 [0/50000 (0%)]	Loss: 0.820652
Train Epoch: 5 

## Excercise 1) RESNET18 모델 구현
- [doc] (https://arxiv.org/pdf/1512.03385.pdf)

![image.png](http://drive.google.com/uc?id=1GgHATI5PFF8-PlBdGp9vDra2maRqLybl)

![image.png](http://drive.google.com/uc?id=1EYxIKJEI0rwIyW7ZZaFeZgJvol6Jb-XL)

![image.png](http://drive.google.com/uc?id=17DgT11woHwXACEGOfvkVd8pYmxRI7vQl)

![image.png](http://drive.google.com/uc?id=12reHf9xtapZrVBG4LlNbNGa37ZeFfUqk)

In [10]:
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from torchvision import transforms, datasets, utils
from torchsummary import summary
import matplotlib.pyplot as plt

# DEVICE 설정
USE_CUDA = torch.cuda.is_available()
DEVICE = torch.device("cuda" if USE_CUDA else "cpu")

# Parameter 설정
EPOCHS = 10
BATCH_SIZE = 64
LR = 0.01

# Transform 설정
transform_CIFAR10 = transforms.Compose([
    # transforms.Resize((224,224)),
    transforms.ToTensor(),
    transforms.Normalize(mean=(0.485, 0.456, 0.406), std=(0.229, 0.224, 0.225)),
])

# Dataset 설정
train_dataset = datasets.CIFAR10(root = '../data',
                                         train = True,
                                         download = True,
                                         transform = transform_CIFAR10)

test_dataset = datasets.CIFAR10(root = '../data',
                                train = False,
                                download = True,
                                transform = transform_CIFAR10)

train_loader = torch.utils.data.DataLoader(dataset = train_dataset,
                                           batch_size = BATCH_SIZE,
                                           shuffle=True)

test_loader = torch.utils.data.DataLoader(dataset = test_dataset,
                                          batch_size = BATCH_SIZE,
                                          shuffle=True)

Files already downloaded and verified
Files already downloaded and verified


In [25]:
# Model 구현
class Custom_RESNET(nn.Module):
    def __init__(self):
        super(Custom_RESNET, self).__init__()
        self.maxpool2d = nn.MaxPool2d(kernel_size=3, stride=2)
        self.relu = nn.ReLU(inplace=True)

        self.conv1 = nn.Conv2d(in_channels=3, out_channels=64, kernel_size=7, stride=2, padding=1)
        self.bn1 = nn.BatchNorm2d(num_features=64)

        self.conv2_1 = nn.Conv2d(64,64,3,padding=1)
        self.conv2_2 = nn.Conv2d(64,64,3,padding=1)
        self.conv2_3 = nn.Conv2d(64,64,3,padding=1)
        self.conv2_4 = nn.Conv2d(64,64,3,padding=1)
        self.bn2_1 = nn.BatchNorm2d(num_features=64)
        self.bn2_2 = nn.BatchNorm2d(num_features=64)
        self.bn2_3 = nn.BatchNorm2d(num_features=64)
        self.bn2_4 = nn.BatchNorm2d(num_features=64)

        self.conv3_1 = nn.Conv2d(64,128,3,padding=1, stride=2)
        self.conv3_2 = nn.Conv2d(128,128,3,padding=1)
        self.conv3_3 = nn.Conv2d(128,128,3,padding=1)
        self.conv3_4 = nn.Conv2d(128,128,3,padding=1)
        self.bn3_1 = nn.BatchNorm2d(num_features=128)
        self.bn3_2 = nn.BatchNorm2d(num_features=128)
        self.bn3_3 = nn.BatchNorm2d(num_features=128)
        self.bn3_4 = nn.BatchNorm2d(num_features=128)

        self.conv4_1 = nn.Conv2d(128,256,3,padding=1, stride=2)
        self.conv4_2 = nn.Conv2d(256,256,3,padding=1)
        self.conv4_3 = nn.Conv2d(256,256,3,padding=1)
        self.conv4_4 = nn.Conv2d(256,256,3,padding=1)
        self.bn4_1 = nn.BatchNorm2d(num_features=256)
        self.bn4_2 = nn.BatchNorm2d(num_features=256)
        self.bn4_3 = nn.BatchNorm2d(num_features=256)
        self.bn4_4 = nn.BatchNorm2d(num_features=256)

        self.conv5_1 = nn.Conv2d(256,512,3,padding=1, stride=2)
        self.conv5_2 = nn.Conv2d(512,512,3,padding=1)
        self.conv5_3 = nn.Conv2d(512,512,3,padding=1)
        self.conv5_4 = nn.Conv2d(512,512,3,padding=1)
        self.bn5_1 = nn.BatchNorm2d(num_features=512)
        self.bn5_2 = nn.BatchNorm2d(num_features=512)
        self.bn5_3 = nn.BatchNorm2d(num_features=512)
        self.bn5_4 = nn.BatchNorm2d(num_features=512)

        self.adaptiveavgpool2d = nn.AdaptiveAvgPool2d(1)

        self.fc = nn.Linear(512, 10)

    def forward(self, x):
        ########################################## Complete This Code~!

        x1 = self.relu(self.bn1(self.conv1(x)))
        x1 = self.maxpool2d(x1)

        x2_1 = self.relu(self.bn2_1(self.conv2_1(x1)))
        x2_2 = self.relu(self.bn2_2(self.conv2_2(x2_1))) + x1
        x2_3 = self.relu(self.bn2_3(self.conv2_3(x2_2)))
        x2_4 = self.relu(self.bn2_4(self.conv2_4(x2_3))) + x2_2

        x3_1 = self.relu(self.bn3_1(self.conv3_1(x2_4)))
        x3_2 = self.relu(self.bn3_2(self.conv3_2(x3_1)))
        x3_3 = self.relu(self.bn3_3(self.conv3_3(x3_2)))
        x3_4 = self.relu(self.bn3_4(self.conv3_4(x3_3))) + x3_2

        x4_1 = self.relu(self.bn4_1(self.conv4_1(x3_4)))
        x4_2 = self.relu(self.bn4_2(self.conv4_2(x4_1)))
        x4_3 = self.relu(self.bn4_3(self.conv4_3(x4_2)))
        x4_4 = self.relu(self.bn4_4(self.conv4_4(x4_3))) + x4_2

        x5_1 = self.relu(self.bn5_1(self.conv5_1(x4_4)))
        x5_2 = self.relu(self.bn5_2(self.conv5_2(x5_1)))
        x5_3 = self.relu(self.bn5_3(self.conv5_3(x5_2)))
        x5_4 = self.relu(self.bn5_4(self.conv5_4(x5_3))) + x5_2

        ########################################## Complete This Code~!

        x = self.adaptiveavgpool2d(x5_4)
        x = x.view(-1,512)
        x = self.fc(x)
        return x

model = Custom_RESNET().to(DEVICE)
summary(model, (3, 32, 32))

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1           [-1, 64, 14, 14]           9,472
       BatchNorm2d-2           [-1, 64, 14, 14]             128
              ReLU-3           [-1, 64, 14, 14]               0
         MaxPool2d-4             [-1, 64, 6, 6]               0
            Conv2d-5             [-1, 64, 6, 6]          36,928
       BatchNorm2d-6             [-1, 64, 6, 6]             128
              ReLU-7             [-1, 64, 6, 6]               0
            Conv2d-8             [-1, 64, 6, 6]          36,928
       BatchNorm2d-9             [-1, 64, 6, 6]             128
             ReLU-10             [-1, 64, 6, 6]               0
           Conv2d-11             [-1, 64, 6, 6]          36,928
      BatchNorm2d-12             [-1, 64, 6, 6]             128
             ReLU-13             [-1, 64, 6, 6]               0
           Conv2d-14             [-1, 6

In [20]:
from torchvision import models
model_import = models.resnet18(pretrained=False, num_classes=10).to(DEVICE)
summary(model_import, (3, 32, 32))

/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=None`.
  warnings.warn(msg)


----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1           [-1, 64, 16, 16]           9,408
       BatchNorm2d-2           [-1, 64, 16, 16]             128
              ReLU-3           [-1, 64, 16, 16]               0
         MaxPool2d-4             [-1, 64, 8, 8]               0
            Conv2d-5             [-1, 64, 8, 8]          36,864
       BatchNorm2d-6             [-1, 64, 8, 8]             128
              ReLU-7             [-1, 64, 8, 8]               0
            Conv2d-8             [-1, 64, 8, 8]          36,864
       BatchNorm2d-9             [-1, 64, 8, 8]             128
             ReLU-10             [-1, 64, 8, 8]               0
       BasicBlock-11             [-1, 64, 8, 8]               0
           Conv2d-12             [-1, 64, 8, 8]          36,864
      BatchNorm2d-13             [-1, 64, 8, 8]             128
             ReLU-14             [-1, 6

In [26]:
# Model, Optimizer 설정
optimizer = optim.SGD(model.parameters(), lr=LR, momentum=0.5)

In [27]:
# Train 구현
def train_one_epoch(model, train_loader, optimizer, epoch):
    model.train()
    for batch_idx, (data, target) in enumerate(train_loader):
        data, target = data.to(DEVICE), target.to(DEVICE)
        optimizer.zero_grad()
        output = model(data)
        loss = F.cross_entropy(output, target)
        loss.backward()
        optimizer.step()

        if batch_idx % 200 == 0:
            print('Train Epoch: {} [{}/{} ({:.0f}%)]\tLoss: {:.6f}'.format(
                epoch, batch_idx * len(data), len(train_loader.dataset),
                100. * batch_idx / len(train_loader), loss.item()))

# Evaluation 구현
def evaluate(model, test_loader):
    model.eval()
    test_loss = 0
    correct = 0
    with torch.no_grad():
        for data, target in test_loader:
            data, target = data.to(DEVICE), target.to(DEVICE)
            output = model(data)

            # 배치 오차를 합산
            test_loss += F.cross_entropy(output, target,
                                         reduction='sum').item()

            # 가장 높은 값을 가진 인덱스가 바로 예측값
            pred = output.max(1, keepdim=True)[1]
            correct += pred.eq(target.view_as(pred)).sum().item()

    test_loss /= len(test_loader.dataset)
    test_accuracy = 100. * correct / len(test_loader.dataset)
    return test_loss, test_accuracy

for epoch in range(1, EPOCHS + 1):
    train_one_epoch(model, train_loader, optimizer, epoch)
    test_loss, test_accuracy = evaluate(model, test_loader)

    print('[{}] Test Loss: {:.4f}, Accuracy: {:.2f}%'.format(
          epoch, test_loss, test_accuracy))

Train Epoch: 1 [0/50000 (0%)]	Loss: 2.668922
Train Epoch: 1 [12800/50000 (26%)]	Loss: 1.611708
Train Epoch: 1 [25600/50000 (51%)]	Loss: 1.521805
Train Epoch: 1 [38400/50000 (77%)]	Loss: 1.354004
[1] Test Loss: 1.3454, Accuracy: 51.97%
Train Epoch: 2 [0/50000 (0%)]	Loss: 0.879050
Train Epoch: 2 [12800/50000 (26%)]	Loss: 1.021214
Train Epoch: 2 [25600/50000 (51%)]	Loss: 0.884004
Train Epoch: 2 [38400/50000 (77%)]	Loss: 1.206973
[2] Test Loss: 1.0645, Accuracy: 62.42%
Train Epoch: 3 [0/50000 (0%)]	Loss: 1.176931
Train Epoch: 3 [12800/50000 (26%)]	Loss: 0.932939
Train Epoch: 3 [25600/50000 (51%)]	Loss: 0.866069
Train Epoch: 3 [38400/50000 (77%)]	Loss: 0.844845
[3] Test Loss: 0.9556, Accuracy: 67.04%
Train Epoch: 4 [0/50000 (0%)]	Loss: 0.632763
Train Epoch: 4 [12800/50000 (26%)]	Loss: 0.893376
Train Epoch: 4 [25600/50000 (51%)]	Loss: 0.634430
Train Epoch: 4 [38400/50000 (77%)]	Loss: 0.615543
[4] Test Loss: 1.0639, Accuracy: 64.88%
Train Epoch: 5 [0/50000 (0%)]	Loss: 1.123219
Train Epoch: 5 

KeyboardInterrupt: ignored